# Attacks in Mexico during elections

_I dataset sono stati prelevati dall'Harvard Dataverse – British Journal of Political Science Dataverse. The bibliography can be found at the end of this notebook._

The motivation for the dataset lies in a surprising wave of attacks characterized by high-profile violence and perpetrated by Mexican drug cartels. These attacks were directed against hundreds of political candidates between 2007 and 2012. 

The researchers suggest that the attacks were perpetrated in an attempt to establish criminal governance, demonstrating the vulnerability of Mexican politics.

## Loading the data

In the file [Dataset_HighProfileCriminalViolence.tab](Dataset_HighProfileCriminalViolence.tab) are listed the number of attacks occurred to elected officials in Mexico between 2007 and 2012.

We will extract Mexican state codes, names, the counts of attacks, and the years when **local** elections occurred, and return a dictionary of dictionaries mapping **two digit** state codes **as strings** to the extracted info.

In [5]:
# Imports

import csv
from pprint import pprint

def load(filename):

    # Declaring the variables that I need to return the correct dictionary
    ret = {}
    id_municipality = ''

    # Opening the file as f
    with open(filename, encoding='utf-8', newline='') as f:

        # Creating a new variable and I assign it a csv DictReader with delimiter='\t' and utf8 encoding
        my_reader = csv.DictReader(f, delimiter='\t')  

        # Adding into the dictionary the codes of each municipality
        for row in my_reader:
            pass

mexico_db = load('Dataset_HighProfileCriminalViolence.tab')
mexico_db

Exception: TODO IMPLEMENT ME !

### Testing

In [ ]:
from expected_mexico_db import expected_mexico_db
from pprint import pformat

for sid in expected_mexico_db.keys():
    if sid not in mexico_db: print('\nERROR: MISSING state', sid); break    
    for k in expected_mexico_db[sid]:
        if k not in mexico_db[sid]:
            print('\nERROR at state', sid,'\n\n   MISSING key:', k); break    
        if expected_mexico_db[sid][k] != mexico_db[sid][k]:
            print('\nERROR at state', sid, 'key:',k)
            print('  ACTUAL:\n', pformat(mexico_db[sid][k]))
            print('  EXPECTED:\n', pformat(expected_mexico_db[sid][k]))
            break

## Attacks visualization

## Cartels visualization